In [ ]:
import os
import urllib.request

import nibabel as nib
from brainbuilder.tests.generate_synthetic_data import generate_synthetic_data


def preprocess_input_data(input_path, output_path):
    """Preprocess the input data by downloading and preparing necessary files."""
    img = nib.load(input_path)
    ar = img.get_fdata()

    # invert intensities
    ar = ar.max() - ar

    # Normalize to 0-255 and convert to uint8
    ar = (ar / ar.max() * 255).astype("uint8")

    # Crop to non-zero region
    # valid_x_indices = (ar.sum(axis=(1,2)) > 0).nonzero()[0]
    valid_y_indices = (ar.sum(axis=(0, 2)) > 0).nonzero()[0]
    # valid_z_indices = (ar.sum(axis=(0,1)) > 0).nonzero()[0]

    ar_cropped = ar[:, valid_y_indices[0] : valid_y_indices[-1] + 1, :]

    print(f"Writing preprocessed data to {output_path}...")
    nib.Nifti1Image(ar_cropped, img.affine).to_filename(output_path)


input_http_path = "https://ftp.bigbrainproject.org/bigbrain-ftp/BigBrainRelease.2015/3D_Volumes/MNI-ICBM152_Space/nii/full16_400um_2009b_sym.nii.gz"

tmp_dir = "/tmp/brainbuilder_test_data"

input_local_path = f"{tmp_dir}/full16_400um_2009b_sym.nii.gz"
preproc_input_path = f"{tmp_dir}/preprocessed_input.nii.gz"

clobber = False

if clobber:
    if os.path.exists(tmp_dir):
        # remove dir, even if it is not empty
        import shutil

        shutil.rmtree(tmp_dir)

    os.makedirs(tmp_dir)

# Download the input file if it does not exist
if not os.path.exists(input_local_path) or clobber:
    print(f"Downloading {input_http_path} to {input_local_path}...")
    urllib.request.urlretrieve(input_http_path, input_local_path)
    print("Download complete.")

if not os.path.exists(preproc_input_path) or clobber:
    preprocess_input_data(input_local_path, preproc_input_path)

generate_synthetic_data(
    input_fn=preproc_input_path,
    out_dir=tmp_dir,
    gm_surf_fn="data/MR1_gray_surface_R_81920.surf.gii",
    wm_surf_fn="data/MR1_white_surface_R_81920.surf.gii",
    clobber=clobber,
)

Download complete.
Writing preprocessed data to /tmp/brainbuilder_test_data/preprocessed_input.nii.gz...
Generating synthetic data
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-12_synth.nii.gz
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-16_synth.nii.gz
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-20_synth.nii.gz
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-24_synth.nii.gz
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-28_synth.nii.gz
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-32_synth.nii.gz
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-36_synth.nii.gz
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-40_synth.nii.gz
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-44_synth.nii.gz
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-48_synth.nii.gz
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-52_synth.nii.gz
/tmp/brainbuilder_test_data/raw_dir//sub-01_

('/tmp/brainbuilder_test_data/sect_info.csv',
 '/tmp/brainbuilder_test_data/chunk_info.csv',
 '/tmp/brainbuilder_test_data/hemi_info.csv')

In [30]:
import pandas as pd

sect_info_csv = f"{tmp_dir}/sect_info.csv"
chunk_info_csv = f"{tmp_dir}/chunk_info.csv"
hemi_info_csv = f"{tmp_dir}/hemi_info.csv"

# Load the sect_info csv and visualize contents. Stores section-wise information
print("Sect Info:")
sect_info_df = pd.read_csv(sect_info_csv)
print(sect_info_df)

# Load the chunk_info csv and visualize contents. Stores chunk-wise information
print("Chunk Info:")
chunk_info_df = pd.read_csv(chunk_info_csv)
print(chunk_info_df)

# Load the hemi_info csv and visualize contents. Stores hemispheric information
print("Hemi Info:")
hemi_info_df = pd.read_csv(hemi_info_csv)
print(hemi_info_df)

Sect Info:
                                                   raw  sub hemisphere  \
0    /tmp/brainbuilder_test_data/raw_dir//sub-01_ch...    1       both   
1    /tmp/brainbuilder_test_data/raw_dir//sub-01_ch...    1       both   
2    /tmp/brainbuilder_test_data/raw_dir//sub-01_ch...    1       both   
3    /tmp/brainbuilder_test_data/raw_dir//sub-01_ch...    1       both   
4    /tmp/brainbuilder_test_data/raw_dir//sub-01_ch...    1       both   
..                                                 ...  ...        ...   
103  /tmp/brainbuilder_test_data/raw_dir//sub-01_ch...    1       both   
104  /tmp/brainbuilder_test_data/raw_dir//sub-01_ch...    1       both   
105  /tmp/brainbuilder_test_data/raw_dir//sub-01_ch...    1       both   
106  /tmp/brainbuilder_test_data/raw_dir//sub-01_ch...    1       both   
107  /tmp/brainbuilder_test_data/raw_dir//sub-01_ch...    1       both   

    acquisition  sample  chunk  
0         synth      12      1  
1         synth      16      1  
2

In [ ]:
%reload_ext autoreload
%autoreload 2

from brainbuilder.reconstruct import reconstruct

resolution_list = [4, 3, 2]

clobber = False

df = reconstruct(
    hemi_info_csv,
    chunk_info_csv,
    sect_info_csv,
    [4, 3, 2, 1],
    f"{tmp_dir}/reconstruction/",
    clobber=clobber,
)

2025-10-14 22:37:10,503 | INFO | Reconstructing 2D sections to 3D volume
2025-10-14 22:37:10,504 | INFO | 	Inputs:
2025-10-14 22:37:10,504 | INFO | 		Hemisphere info: /tmp/brainbuilder_test_data/hemi_info.csv
2025-10-14 22:37:10,505 | INFO | 		Chunk info: /tmp/brainbuilder_test_data/chunk_info.csv
2025-10-14 22:37:10,506 | INFO | 		Section info: /tmp/brainbuilder_test_data/sect_info.csv
2025-10-14 22:37:10,506 | INFO | 		Max 3D resolution: 0.3
2025-10-14 22:37:10,507 | INFO | 		Final resolution: None
2025-10-14 22:37:10,508 | INFO | 		Resolution list: [4, 3, 2, 1]
2025-10-14 22:37:10,508 | INFO | 		Linear steps: ['rigid', 'similarity', 'affine']
2025-10-14 22:37:10,509 | INFO | 		Number of cores: -1
2025-10-14 22:37:10,510 | INFO | 		Segmentation method: nnunetv1
2025-10-14 22:37:10,510 | INFO | 		Use 3D nonlinear CC: True
2025-10-14 22:37:10,511 | INFO | 		Use 2D nonlinear: True
2025-10-14 22:37:10,511 | INFO | 		Missing Section Interpolation method: volumetric
2025-10-14 22:37:10,512

	Downsampling sub: 1, hemisphere: both, chunk: 1
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-12_synth.nii.gz (493, 473) [198. 190.]
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-16_synth.nii.gz (493, 473) [198. 190.]
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-20_synth.nii.gz (493, 473) [198. 190.]
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-24_synth.nii.gz (493, 473) [198. 190.]
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-32_synth.nii.gz (493, 473) [198. 190.]
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-28_synth.nii.gz (493, 473) [198. 190.]
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-56_synth.nii.gz (493, 473) /tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-72_synth.nii.gz (493, 473) [198. 190.]
[198. 190.]
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-36_synth.nii.gz (493, 473) [198. 190.]
/tmp/brainbuilder_test_data/raw_dir//sub-01_chunk-1_sample-44_synth.n

2025-10-14 22:37:12,358 | INFO | Wrote: /tmp/brainbuilder_test_data/reconstruction//1_seg//nnunet/sub-01_chunk-1_sample-68_synth_1mm_0000.nii.gz
2025-10-14 22:37:12,364 | INFO | Wrote: /tmp/brainbuilder_test_data/reconstruction//1_seg//nnunet/sub-01_chunk-1_sample-108_synth_1mm_0000.nii.gz
2025-10-14 22:37:12,369 | INFO | Wrote: /tmp/brainbuilder_test_data/reconstruction//1_seg//nnunet/sub-01_chunk-1_sample-12_synth_1mm_0000.nii.gz
2025-10-14 22:37:12,369 | INFO | Wrote: /tmp/brainbuilder_test_data/reconstruction//1_seg//nnunet/sub-01_chunk-1_sample-16_synth_1mm_0000.nii.gz
2025-10-14 22:37:12,369 | INFO | Wrote: /tmp/brainbuilder_test_data/reconstruction//1_seg//nnunet/sub-01_chunk-1_sample-20_synth_1mm_0000.nii.gz
2025-10-14 22:37:12,370 | INFO | Wrote: /tmp/brainbuilder_test_data/reconstruction//1_seg//nnunet/sub-01_chunk-1_sample-28_synth_1mm_0000.nii.gz
2025-10-14 22:37:12,370 | INFO | Wrote: /tmp/brainbuilder_test_data/reconstruction//1_seg//nnunet/sub-01_chunk-1_sample-24_synth_

results_folder /home/tfunck/projects/brainbuilder/brainbuilder/..//nnUNet//../


Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /home/tfunck/projects/brainbuilder/brainbuilder/..//nnUNet//../nnUNet/2d/Task502_cortex/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 1 input modalities for each image
Found 108 unique case ids, here are some examples: ['sub-01_chunk-1_sample-88_synth_1mm'
 'sub-01_chunk-1_sample-264_synth_1mm'
 'sub-01_chunk-1_sample-140_synth_1mm'
 'sub-01_chunk-1_sample-216_synth_1mm'
 'sub-01_chunk-1_sample-304_synth_1mm'
 'sub-01_chunk-1_sample-108_synth_1mm'
 'sub-01_chunk-1_sample-96_synth_1mm'
 'sub-01_chunk-1_sample-436_synth_1mm'
 

In [ ]:
# Visualize results

